In [3]:
import pandas as pd
import csv
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image


# Cleaning the Data that was collected. This function will go through all of the bazaar data that has been saved, Determine a rough "reliability" of each product and format it to be ready for use in our Model.


In [11]:
def cleanBazaarData():
    with open("bazaarData.txt", 'r') as file:
        reader = csv.reader(file, delimiter=';')
        dataset = {}
        n = 0



        for line in reader:
            di = eval(line[1])
            di.update({'product': line[0]})

            reliability = 0
            volumePerc = (di['sellMovingWeek'] / (
                        di['buyMovingWeek'] + di['sellMovingWeek'])) >= .40
            boolWeekly = di['sellMovingWeek'] >= 150000 and di['buyMovingWeek'] >= 150000
            boolOrder = di['sellOrders'] >= 40 and di['buyOrders'] >= 40
            # Determine Weights (A lot of trial and error)

            if di['RoI'] >= 5:
                reliability += 3
                if di['margin'] >= 40:
                    reliability += 3
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                elif 15 < di['margin'] < 40:
                    reliability += 2
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2

            elif 0 < di['RoI'] < 5:
                reliability += 1
                if di['margin'] >= 40:
                    reliability += 3
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                elif 15 < di['margin'] < 40:
                    reliability += 2
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2

            else:
                # print(product)
                continue
            di['reliability'] = reliability
            dataset.update({n: di})
            n += 1

        for k in dataset:
            if dataset[k]['reliability'] >= 6:
                dataset[k]['reliability'] = 1
            else:
                dataset[k]['reliability'] = 0




        df = pd.DataFrame.from_dict(dataset,orient='index', columns=['product','buyVolume', 'buyOrders', 'buyPrice', 'buyMovingWeek', 'sellVolume',
                                                      'sellOrders', 'sellPrice', 'sellMovingWeek', 'RoI', 'margin','buySellDiff','reliability'])

        cleanDF = df.drop(columns=['product'])

        

        return cleanDF.drop_duplicates()


data = cleanBazaarData()


# Dropping the "reliability" predictions from the dataset after storing the feature in 'y' so that we can create a valid train test split we then implement our Random Forest Classifier, the parameters were tweaked for a while, but as expected the more trees the better when dealing with data with lots of possible "number" outcomes.

In [10]:
y = data['reliability']
X = data.drop(columns=['reliability'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)


RFclass = RandomForestClassifier(max_depth=5, random_state=0, criterion='gini', n_estimators=75, max_features=4, bootstrap=True)
RFclass.fit(X_train, y_train)
pred = RFclass.predict(X_test)

RFclassCFM = confusion_matrix(y_test, pred)
print(round((RFclassCFM[0][0]/(RFclassCFM[0][0]+RFclassCFM[1][0]))*100, 2), '% Correct Predictions. Random Forest Classifier')



98.78 % Correct Predictions. Random Forest Classifier


## The commented out section is how we save our Machine Learning model so that it can be used in future projects if we consider the predictions to be viable enough After that I take just one tree and post the image of it, so that we can see an example of a tree created based on our dataset.

In [13]:
#filename = 'finalized_model.sav'
#pickle.dump(RFclass, open(filename, 'wb'))
estimators = RFclass.estimators_

for k in  range(1):
    export_graphviz(estimators[k], out_file='tree'+str(k)+'.dot',
                    feature_names=X.columns,
                    class_names=[str(0),str(1)],
                    rounded=True, proportion=False,
                    precision=2, filled=True)
    call(['dot', '-Tpng', 'tree'+str(k)+'.dot', '-o', 'tree'+str(k)+'.png', '-Gdpi=600'])



In [ ]:
# Control+Mousewheel up to zoom in on the image :D this is 1 out of the 75 trees.
# What happens is all trees give a final predictions and the final predictions will be based off of popularity.

In [14]:
Image(filename='tree0.png') 